In [ ]:
"hi"

In [ ]:
%pwd

In [ ]:
import os

os.chdir("../")

In [ ]:
%pwd

# RAG Chatbot architecture building

1. Load the documents
2. Preprocess the document (remove the metadta and just keep the page_source and page_content)
3. Chunk the data
4. Convert the chunks into embeddings

## 1. Load the PDF file

In [ ]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from typing import List

def load_documents(data_path: str) -> List:
    loader = DirectoryLoader(
        data_path, glob="*.pdf", loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents


In [ ]:
extracted_documents = load_documents("data")
extracted_documents

In [ ]:
len(extracted_documents)

### Proprocess the extracted documents

In [ ]:
from langchain.schema import Document
from typing import List

def filter_minimal_docs(docs: List[Document]) -> List[Document]:
    # filer out the docs that are two short

    minimal_docs = []

    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [ ]:
minimal_docs = filter_minimal_docs(extracted_documents)
minimal_docs

In [ ]:
len(minimal_docs)

## 2. Chunking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split the data into smaller chunks

def chunk_data(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunk = text_splitter.split_documents(minimal_docs)
    return text_chunk


In [ ]:
chunked_data = chunk_data(minimal_docs)
chunked_data

In [ ]:
print(f'number of chunks: {len(chunked_data)}')

# Convert chunks into embeddings

In [ ]:
# Download embeddings model from the Hugging face (sentence transformer)
from langchain.embeddings import HuggingFaceBgeEmbeddings

def download_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_name
    )
    return embeddings

In [ ]:
embedding = download_embeddings()

In [ ]:
embedding

In [ ]:
vector = embedding.embed_query("Hi, How are you?")
vector

In [ ]:
print(f'The dimension of the vector is: {len(vector)}')

# Setup pinecone Vector db

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_api_key

In [ ]:
from pinecone import Pinecone

# create a pinecone client
pc = Pinecone(api_key=pinecone_api_key)
print(f'Pinecone client: {pc}')


In [ ]:
from pinecone import ServerlessSpec


index_name = "medical-chatbot"

if not pc.has_index:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


# create index
pc.Index(index_name)